<a href="https://colab.research.google.com/github/joey0629/chatWeb/blob/master/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#Colab資料夾設定方法
#!pip install google.colab #如未安裝取消註解後執行
import os

from google.colab import drive
drive.mount('/content/drive')

##出現提示欄進行授權

os.chdir('/content/drive/MyDrive/CNN') #切換該目錄
os.listdir() #確認目錄內容

Mounted at /content/drive


['model_2.weights.h5', 'model.weights.h5', 'DL.ipynb']

# 以下為DL model


In [2]:
!pip install captcha
!pip install --upgrade tensorflow
!pip install matplotlib
!pip install --upgrade keras
#!pip install --upgrade deepsegment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 40.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
    

In [9]:
!pip install -U tensorflow keras  # Keras-CV needs TF 2.15 when used with Keras 3
!pip install -U keras-cv
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import keras_cv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 12.8 MB/s eta 0:00:00


In [10]:
import keras
print(keras.__version__)

3.3.3


In [11]:
# 0-9，A-Z，总共36个类别
# characters = string.digits + string.ascii_uppercase
# print(characters)
characters = "0123456789ABCDEFGHJKLMNPQRSTUVWXYZ"
width, height, n_len, n_class = 170, 80, 4, len(characters)
n_classes = len(characters)
print(n_classes)

34


In [12]:
# -*- coding: utf-8 -*-
from captcha.image import ImageCaptcha
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
#from tensorflow.keras.utils.np_utils import to_categorical

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications import VGG16
import string
from tqdm import tqdm

def gen(batch_size=16, height=80, width=170, n_len=4, n_class=34):
    """
    生成器，默认每次生成16张验证码样本
    :param batch_size: 每次生成验证码个数
    :param height: 图片高度
    :param width: 图片宽度
    :param n_len: 验证码中数字/字母的个数
    :param n_class: 类别数

    :yield:
        :X: 图片数据，of shape (batch_size, height, width, 3)
        :y: one-hot 标签数据, of shape (batch_size, n_len, n_class)
    """
    # characters = string.digits + string.ascii_uppercase
    characters = "0123456789ABCDEFGHJKLMNPQRSTUVWXYZ"
    X = np.zeros((batch_size, height, width, 3), dtype=np.uint8)
    y = np.zeros((batch_size, n_len, n_class), dtype=np.uint8)
    generator = ImageCaptcha(width=width, height=height)

    while True:
        for i in range(batch_size):
            random_str = ''.join([random.choice(characters) for j in range(n_len)])
            X[i] = generator.generate_image(random_str)
            for j, ch in enumerate(random_str):
                y[i, j, characters.find(ch)] = 1
        yield X, y


def decode(y):
    """
    将模型输出的 one-hot 解码成相应的4个数字/字母
    :param y: one-hot
    :return:
    """
    # characters = string.digits + string.ascii_uppercase
    characters = "0123456789ABCDEFGHJKLMNPQRSTUVWXYZ"
    y = np.argmax(np.array(y), axis=2)[:,0]
    return ''.join([characters[x] for x in y])



def model(height, width):
    """
    神经网络模型
    :param height: 图片高度
    :param width: 图片宽度
    :return:
    """
    input_tensor = keras.src.Input((height, width, 3))
    base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

    x = base_model.output
    x = keras.layers.Flatten()(x)
    x = Dropout(0.25)(x)
    x = [Dense(n_class, activation='softmax', name='c%d' % (i + 1))(x) for i in range(4)]
    x = keras.layers.Concatenate(axis=1)(x)  # Concatenate the outputs
    x = keras.layers.Reshape((4, n_class))(x)  # Reshape the output
    model = keras.src.Model(inputs=base_model.input, outputs=x)
    # model.load_weights('/content/drive/MyDrive/CNN/model.weights.h5')
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    # Create a tf.data.Dataset from the generator
    train_dataset = tf.data.Dataset.from_generator(
        lambda: gen(),
        output_signature=(
            tf.TensorSpec(shape=(None, height, width, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 4, n_class), dtype=tf.float32)  # Adjust the target shape
        )
    ).prefetch(tf.data.AUTOTUNE)

    validation_dataset = tf.data.Dataset.from_generator(
        lambda: gen(),
        output_signature=(
            tf.TensorSpec(shape=(None, height, width, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 4, n_class), dtype=tf.float32)  # Adjust the target shape
        )
    ).prefetch(tf.data.AUTOTUNE)

    # Use the dataset in the fit method
    model.fit(train_dataset, steps_per_epoch=64, epochs=5,
              validation_data=validation_dataset, validation_steps=1280)

    # 保存模型权重
    model.save_weights("/content/drive/MyDrive/CNN/model_2.weights.h5")


    return model

def evaluate(model, batch_num=200):
    """
    计算模型的总体准确率
    :param model:
    :param batch_num:
    :return:
    """
    batch_acc = 0
    generator = gen()
    for i in tqdm(range(batch_num)):
        X, y = next(generator)
        y_pred = model.predict(X)
        y_pred = np.argmax(y_pred, axis=2).T
        y_true = np.argmax(y, axis=2).T
        batch_acc += np.mean(list(map(np.array_equal, y_true, y_pred)))

    return batch_acc / batch_num



In [5]:
def test_model(model, num_tests=10):
    """
    測試模型的預測結果
    :param model: 要測試的模型
    :param num_tests: 要進行的測試數量
    :return: None
    """
    characters = string.digits + string.ascii_uppercase
    width, height, n_len, n_class = 170, 80, 4, len(characters)
    generator = gen2(1, height, width, n_len, n_class)

    for i in range(num_tests):
        X, y_true = next(generator)
        y_pred = model.predict(X)
        y_true = decode(y_true)
        y_pred = decode(y_pred)

        if y_true == y_pred:
            print(f"Test {i+1}: Success! True = {y_true}, Predicted = {y_pred}")
        else:
            print(f"Test {i+1}: Fail. True = {y_true}, Predicted = {y_pred}")


In [6]:
from keras.models import load_model
# 定義輸入張量
input_tensor = Input(shape=(height, width, 3))

# 加載預訓練的VGG16模型，不包括頂部的全連接層
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

# 獲取VGG16的輸出
x = base_model.output
x = Flatten()(x)  # 展平特徵圖
x = Dropout(0.25)(x)  # 添加Dropout層

# 為每個類別創建獨立的全連接層，然後將它們連接在一起
x = [Dense(n_class, activation='softmax', name='c%d' % (i + 1))(x) for i in range(4)]
x = Concatenate(axis=1)(x)  # 將輸出連接起來
x = Reshape((4, n_class))(x)  # 將輸出重塑為(4, n_class)的形狀

# 定義最終的模型
model = Model(inputs=base_model.input, outputs=x)

# 檢查模型架構
model.summary()

# 加載已經訓練好的模型權重
# model.load_weights('/content/drive/MyDrive/CNN/model.weights.h5')

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 80, 170, 3)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1 (Conv2D)     │ (None, 80, 170, 64)    │          1,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2 (Conv2D)     │ (None, 80, 170, 64)    │         36,928 │ block1_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_pool               │ (None, 40, 85, 64)     │              0 │ block1_conv2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_conv1 (Conv2D)     │ (None, 40, 85, 128)    │         73,856 │ block1_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_conv2 (Conv2D)     │ (None, 40, 85, 128)    │        147,584 │ block2_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_pool               │ (None, 20, 42, 128)    │              0 │ block2_conv2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_conv1 (Conv2D)     │ (None, 20, 42, 256)    │        295,168 │ block2_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_conv2 (Conv2D)     │ (None, 20, 42, 256)    │        590,080 │ block3_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_conv3 (Conv2D)     │ (None, 20, 42, 256)    │        590,080 │ block3_conv2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_pool               │ (None, 10, 21, 256)    │              0 │ block3_conv3[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_conv1 (Conv2D)     │ (None, 10, 21, 512)    │      1,180,160 │ block3_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_conv2 (Conv2D)     │ (None, 10, 21, 512)    │      2,359,808 │ block4_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_conv3 (Conv2D)     │ (None, 10, 21, 512)    │      2,359,808 │ block4_conv2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_pool               │ (None, 5, 10, 512)     │              0 │ block4_conv3[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block5_conv1 (Conv2D)     │ (None, 5, 10, 512)     │      2,359,808 │ block4_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block5_conv2 (Conv2D)     │ (None, 5, 10, 512)     │      2,359,808 │ block5_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block5_conv3 (Conv2D)

 Total params: 15,411,144 (58.79 MB)

 Trainable params: 15,411,144 (58.79 MB)

 Non-trainable params: 0 (0.00 B)

# CNN VGG 16 with normalization and dropout

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout, BatchNormalization, Concatenate, Reshape
from tensorflow.keras.models import Model

def vgg16_with_batchnorm_dropout(input_shape=(80, 170, 3), n_class=34, dropout_rate=0.25):
    """
    構建帶有BatchNormalization和Dropout的VGG16模型
    :param input_shape: 輸入圖像的形狀
    :param n_class: 類別數
    :param dropout_rate: Dropout的比例
    :return: 構建好的模型
    """
    input_tensor = Input(shape=input_shape)

    # Block 1
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(input_tensor)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 2
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 3
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 4
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 5
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Flatten and Fully Connected Layers
    x = Flatten()(x)
    x = Dropout(dropout_rate)(x)
    x = [Dense(n_class, activation='softmax', name='c%d' % (i + 1))(x) for i in range(4)]
    x = Concatenate(axis=1)(x)
    x = Reshape((4, n_class))(x)

    # Create Model
    model = Model(inputs=input_tensor, outputs=x)

    # Compile Model
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

    return model

# 使用生成器進行訓練
def gen2(batch_size=16, height=80, width=170, n_len=4, n_class=36):
    """
    生成器，默认每次生成16张验证码样本
    :param batch_size: 每次生成验证码个数
    :param height: 图片高度
    :param width: 图片宽度
    :param n_len: 验证码中数字/字母的个数
    :param n_class: 类别数
    :yield: 生成的图片数据和标签数据
    """
    from captcha.image import ImageCaptcha
    import string
    import random
    import numpy as np

    characters = string.digits + string.ascii_uppercase

    X = np.zeros((batch_size, height, width, 3), dtype=np.uint8)
    y = np.zeros((batch_size, n_len, n_class), dtype=np.uint8)
    generator = ImageCaptcha(width=width, height=height)

    while True:
        for i in range(batch_size):
            random_str = ''.join([random.choice(characters) for j in range(n_len)])
            X[i] = generator.generate_image(random_str)
            for j, ch in enumerate(random_str):
                y[i, j, characters.find(ch)] = 1
        yield X, y

# 構建和訓練模型
model = vgg16_with_batchnorm_dropout(input_shape=(80, 170, 3), n_class=36, dropout_rate=0.25)

train_dataset = tf.data.Dataset.from_generator(
    lambda: gen2(),
    output_signature=(
        tf.TensorSpec(shape=(None, 80, 170, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 4, 36), dtype=tf.float32)
    )
).prefetch(tf.data.AUTOTUNE)

validation_dataset = tf.data.Dataset.from_generator(
    lambda: gen2(),
    output_signature=(
        tf.TensorSpec(shape=(None, 80, 170, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 4, 36), dtype=tf.float32)
    )
).prefetch(tf.data.AUTOTUNE)

model.fit(train_dataset, steps_per_epoch=5120, epochs=5, validation_data=validation_dataset, validation_steps=1280)

# 保存模型权重
model.save_weights("/content/drive/MyDrive/CNN/model.weights.h5")


Epoch 1/5
5120/5120 ━━━━━━━━━━━━━━━━━━━━ 847s 155ms/step - accuracy: 0.0256 - loss: 193.5060 - val_accuracy: 6.2256e-04 - val_loss: 126.9678
Epoch 2/5
5120/5120 ━━━━━━━━━━━━━━━━━━━━ 789s 154ms/step - accuracy: 0.0275 - loss: 158.3352 - val_accuracy: 0.0023 - val_loss: 130.0143
Epoch 3/5
5120/5120 ━━━━━━━━━━━━━━━━━━━━ 792s 155ms/step - accuracy: 0.0307 - loss: 149.4108 - val_accuracy: 0.0107 - val_loss: 130.8950
Epoch 4/5
5120/5120 ━━━━━━━━━━━━━━━━━━━━ 788s 154ms/step - accuracy: 0.0318 - loss: 142.1754 - val_accuracy: 0.0083 - val_loss: 131.0560
Epoch 5/5
5120/5120 ━━━━━━━━━━━━━━━━━━━━ 787s 154ms/step - accuracy: 0.0319 - loss: 132.8136 - val_accuracy: 0.0115 - val_loss: 129.7426


FileNotFoundError: [Errno 2] Unable to synchronously create file (unable to open file: name = '/content/drive/MyDrive/CNN/model.weights.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

# CNN VGG 16 Github


In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from PIL import Image
import numpy as np
import csv
print("Creating CNN model...")
lin = Input((60, 200, 3))
out = lin
out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Flatten()(out)
out = Dropout(0.3)(out)
out = [Dense(34, name='digit1', activation='softmax')(out),\
    Dense(34, name='digit2', activation='softmax')(out),\
    Dense(34, name='digit3', activation='softmax')(out),\
    Dense(34, name='digit4', activation='softmax')(out),\
    Dense(34, name='digit5', activation='softmax')(out)]
model = Model(inputs=lin, outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Creating CNN model...


Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 60, 200, 3)     │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 60, 200, 32)    │            896 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 58, 198, 32)    │          9,248 │ conv2d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 58, 198, 32)    │            128 │ conv2d_8[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 29, 99, 32)     │              0 │ batch_normalization_4… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_11 (Dropout)      │ (None, 29, 99, 32)     │              0 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 29, 99, 64)     │         18,496 │ dropout_11[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_10 (Conv2D)        │ (None, 27, 97, 64)     │         36,928 │ conv2d_9[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_5     │ (None, 27, 97, 64)     │            256 │ conv2d_10[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 13, 48, 64)     │              0 │ batch_normalization_5… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_12 (Dropout)      │ (None, 13, 48, 64)     │              0 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_11 (Conv2D)        │ (None, 13, 48, 128)    │         73,856 │ dropout_12[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_12 (Conv2D)        │ (None, 11, 46, 128)    │        147,584 │ conv2d_11[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_6     │ (None, 11, 46, 128)    │            512 │ conv2d_12[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_6           │ (None, 5, 23, 128)     │              0 │ batch_normalization_6… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_13 (Dropout)      │ (None, 5, 23, 128)     │              0 │ max_pooling2d_6[0][0]  │
├──────────────────────

 Total params: 1,019,466 (3.89 MB)

 Trainable params: 1,018,506 (3.89 MB)

 Non-trainable params: 960 (3.75 KB)

# Model Running

In [ ]:
model = vgg16_with_batchnorm_dropout(input_shape=(80, 170, 3), n_class=36, dropout_rate=0.25)
model.summary()

In [ ]:
# model = model(height, width)
# 使用模型進行測試
model = model.load_weights('/content/drive/MyDrive/CNN/model.weights.h5')
# test_model(model)
X, y = next(gen(1))

# 测试一个样本
y_pred = model.predict(X)
plt.title('real: %s\npred:%s' % (decode(y), decode(y_pred)))
plt.imshow(X[0], cmap='gray')
plt.show()

print("模型总体准确率：", evaluate(model))

AttributeError: 'NoneType' object has no attribute 'predict'